## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Belaya Gora,RU,2022-03-10 15:33:44,68.5333,146.4167,-16.24,100,100,2.62,overcast clouds
1,1,Tautira,PF,2022-03-10 15:33:45,-17.7333,-149.1500,80.74,72,91,12.12,overcast clouds
2,2,Avera,US,2022-03-10 15:33:45,33.1940,-82.5271,56.71,95,100,3.91,light rain
3,3,Punta Arenas,CL,2022-03-10 15:33:46,-53.1500,-70.9167,50.11,53,0,8.05,clear sky
4,4,Liku,ID,2022-03-10 15:33:46,-0.7275,119.8922,77.45,91,77,3.18,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Desired minimum temperature for your trip? "))
max_temp = float(input("Desired maximum temperature for your trip? "))

Desired minimum temperature for your trip? 30
Desired maximum temperature for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pre_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
pre_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Tautira,PF,2022-03-10 15:33:45,-17.7333,-149.1500,80.74,72,91,12.12,overcast clouds
2,2,Avera,US,2022-03-10 15:33:45,33.1940,-82.5271,56.71,95,100,3.91,light rain
3,3,Punta Arenas,CL,2022-03-10 15:33:46,-53.1500,-70.9167,50.11,53,0,8.05,clear sky
4,4,Liku,ID,2022-03-10 15:33:46,-0.7275,119.8922,77.45,91,77,3.18,broken clouds
5,5,Rikitea,PF,2022-03-10 15:33:47,-23.1203,-134.9692,78.42,69,79,6.87,broken clouds
7,7,Quelimane,MZ,2022-03-10 15:33:48,-17.8786,36.8883,85.95,70,40,17.27,scattered clouds
8,8,Castro,BR,2022-03-10 15:33:48,-24.7911,-50.0119,75.69,80,53,5.91,broken clouds
9,9,Albany,US,2022-03-10 15:32:07,42.6001,-73.9662,36.95,88,21,2.80,few clouds
10,10,Alma,CA,2022-03-10 15:33:25,48.5501,-71.6491,33.42,78,96,11.77,overcast clouds
12,12,Busselton,AU,2022-03-10 15:33:49,-33.6500,115.3333,71.49,52,48,9.89,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
pre_cities_df.isnull().sum()

City_ID        0
City           0
Country        3
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pre_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Tautira,PF,80.74,overcast clouds,-17.7333,-149.1500,
2,Avera,US,56.71,light rain,33.1940,-82.5271,
3,Punta Arenas,CL,50.11,clear sky,-53.1500,-70.9167,
4,Liku,ID,77.45,broken clouds,-0.7275,119.8922,
5,Rikitea,PF,78.42,broken clouds,-23.1203,-134.9692,
7,Quelimane,MZ,85.95,scattered clouds,-17.8786,36.8883,
8,Castro,BR,75.69,broken clouds,-24.7911,-50.0119,
9,Albany,US,36.95,few clouds,42.6001,-73.9662,
10,Alma,CA,33.42,overcast clouds,48.5501,-71.6491,
12,Busselton,AU,71.49,scattered clouds,-33.6500,115.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found..")
        

Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..
Hotel not found..


In [9]:
# 7. Drop the rows where there is no Hotel Name.
found_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
found_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Tautira,PF,80.74,overcast clouds,-17.7333,-149.1500,Pension Oaoa
3,Punta Arenas,CL,50.11,clear sky,-53.1500,-70.9167,Hotel Hain
4,Liku,ID,77.45,broken clouds,-0.7275,119.8922,Wahyu
5,Rikitea,PF,78.42,broken clouds,-23.1203,-134.9692,People ThankYou
7,Quelimane,MZ,85.95,scattered clouds,-17.8786,36.8883,Hotel Flamingo


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
found_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in found_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = found_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))